In [11]:
import os
import pandas as pd
from pandas import DataFrame
from datetime import datetime as dt
import matplotlib.pyplot as plt

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine, inspect

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Boolean 

from sqlalchemy.orm import Session

In [12]:
engine = create_engine("sqlite:///../Data/megawatt.sqlite")

In [13]:
newdf = DataFrame(engine.execute("SELECT * FROM demand where date >'2016-01-01'and date < '2018-12-01' and name='ISNE'").fetchall())
newdf = newdf.rename(columns={
    newdf.columns[0]: 'name',
    newdf.columns[1]: 'datetime',
    newdf.columns[2]: 'value',
    newdf.columns[3] : 'date',
    newdf.columns[4] : 'hour'
    })

newdf['datetime'] = pd.to_datetime(newdf['datetime'])
# newdf.sort_values('datetime',ascending=True)
newdf.head()


,name,datetime,value,date,hour
0,ISNE,2018-10-17 02:00:00,13130.0,2018-10-17,2
1,ISNE,2018-10-17 01:00:00,14055.0,2018-10-17,1
2,ISNE,2018-10-17 00:00:00,14683.0,2018-10-17,0
3,ISNE,2018-10-16 23:00:00,14698.0,2018-10-16,23
4,ISNE,2018-10-16 22:00:00,13792.0,2018-10-16,22


In [14]:
weather_file = os.path.join('../Data','2016-2018_MA_weather_data.csv')

In [15]:
raw_listings_df = pd.read_csv(weather_file, encoding="ISO-8859-1")

In [16]:
raw_listings_df['datetime'] = pd.to_datetime(raw_listings_df['datetime'])
raw_listings_df.head()

,Unnamed: 0,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf,datetime
0,0,725090,14739,270,57.0,99999,5.6,28.04,42.08,2016-01-01 00:00:00
1,1,725090,14739,260,51.0,22000,5.0,28.04,41.00,2016-01-01 01:00:00
2,2,725090,14739,260,62.0,1433,5.0,26.96,41.00,2016-01-01 02:00:00
3,3,725090,14739,260,46.0,1433,4.4,26.96,39.92,2016-01-01 03:00:00
4,5,725090,14739,260,57.0,1311,4.4,26.96,39.92,2016-01-01 04:00:00


In [17]:
del raw_listings_df['Unnamed: 0']

In [18]:
raw_listings_df.head()

,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf,datetime
0,725090,14739,270,57.0,99999,5.6,28.04,42.08,2016-01-01 00:00:00
1,725090,14739,260,51.0,22000,5.0,28.04,41.00,2016-01-01 01:00:00
2,725090,14739,260,62.0,1433,5.0,26.96,41.00,2016-01-01 02:00:00
3,725090,14739,260,46.0,1433,4.4,26.96,39.92,2016-01-01 03:00:00
4,725090,14739,260,57.0,1311,4.4,26.96,39.92,2016-01-01 04:00:00


In [19]:
whatdf = pd.merge(newdf, raw_listings_df, on='datetime',how='inner')
whatdf.head()

,name,datetime,value,date,hour,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf
0,ISNE,2018-10-14 05:00:00,10443.0,2018-10-14,5,725090,14739,999,NaN,99999,NaN,NaN,NaN
1,ISNE,2018-10-14 04:00:00,10884.0,2018-10-14,4,725090,14739,310,31.0,22000,8.3,39.92,46.94
2,ISNE,2018-10-14 03:00:00,11436.0,2018-10-14,3,725090,14739,290,41.0,22000,9.4,39.92,48.92
3,ISNE,2018-10-14 02:00:00,12203.0,2018-10-14,2,725090,14739,280,46.0,22000,10.6,39.92,51.08
4,ISNE,2018-10-14 01:00:00,12774.0,2018-10-14,1,725090,14739,280,31.0,22000,10.6,39.92,51.08


In [20]:
whatdf.to_csv('out.csv')